In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import models, layers
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.models import Model
from keras.preprocessing.image import array_to_img
from keras.callbacks import Callback
import numpy as np
import os

## Import Libraries

In [2]:
IMG_WIDTH = 178
IMG_HEIGHT = 218
BATCH_SIZE = 32

In [5]:
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from keras.utils.vis_utils import plot_model
from matplotlib import image
from matplotlib import pyplot as plt
import os
import time
import tensorflow as tf
from tensorflow import keras
tf.config.run_functions_eagerly(True)

# The batch size we'll use for training
batch_size = 64

# These many images will be used from the data archive
dataset_split = 2500

master_dir = 'data'
x = []
y = []
for image_file in os.listdir( master_dir )[ 0 : dataset_split ]:
    rgb_image = Image.open( os.path.join( master_dir , image_file ) ).resize( ( IMG_HEIGHT , IMG_WIDTH ) )
    # Normalize the RGB image array
    rgb_img_array = (np.asarray( rgb_image ) ) / 255
    gray_image = rgb_image.convert( 'L' )
    # Normalize the grayscale image array
    gray_img_array = ( np.asarray( gray_image ).reshape( ( IMG_HEIGHT , IMG_WIDTH , 1 ) ) ) / 255
    # Append both the image arrays
    x.append( gray_img_array )
    y.append( rgb_img_array )

# Train-test splitting
train_x, test_x, train_y, test_y = train_test_split( np.array(x) , np.array(y) , test_size=0.1 )

# Construct tf.data.Dataset object
dataset = tf.data.Dataset.from_tensor_slices( ( train_x , train_y ) )
dataset = dataset.batch( batch_size )


ModuleNotFoundError: No module named 'keras.utils.vis_utils'

In [ ]:
data_dir = 'data/img_align_celeba'
image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir)]

def convert_to_rgb(image):
    # Assuming image is grayscale with shape [IMG_HEIGHT, IMG_WIDTH, 1]
    return tf.image.grayscale_to_rgb(image)  # Converts grayscale image to RGB

def preprocess_image(image):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1.0  # Normalize to [-1, 1]
    return image

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.cond(
        tf.strings.regex_full_match(image_path, '.*\\.jpg$'),
        lambda: tf.image.decode_jpeg(image, channels=3),
        lambda: tf.image.decode_png(image, channels=3)
    )
    image = preprocess_image(image)
    return image

# Dataset preparation
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(lambda x: load_image(x))
dataset = dataset.batch(BATCH_SIZE).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
data_dir = 'data/img_align_celeba'
image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir)]

def convert_to_rgb(image):
    # Assuming image is grayscale with shape [IMG_HEIGHT, IMG_WIDTH, 1]
    return tf.image.grayscale_to_rgb(image)  # Converts grayscale image to RGB

def preprocess_image(image):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1.0  # Normalize to [-1, 1]
    return image

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.cond(
        tf.strings.regex_full_match(image_path, '.*\\.jpg$'),
        lambda: tf.image.decode_jpeg(image, channels=3),
        lambda: tf.image.decode_png(image, channels=3)
    )
    image = preprocess_image(image)
    return image

# Dataset preparation
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(lambda x: load_image(x))
dataset = dataset.batch(BATCH_SIZE).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Split the data

In [ ]:
def train_test_split(dataset, train_size=0.8):
    # dataset = dataset.shuffle(buffer_size=10000, seed=42)
    
    dataset_size = len(dataset)
    train_size = int(dataset_size * train_size)
    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)
    return train_dataset, test_dataset

In [ ]:
train_dataset, test_dataset = train_test_split(dataset)

In [ ]:
print(f"Training dataset: {len(train_dataset)} batches")
print(f"Test dataset: {len(test_dataset)} batches")

In [ ]:
train_dataset.as_numpy_iterator().next()

## Resize and Normalize the Images

In [ ]:
train_dataset = train_dataset.map(
    lambda x: tf.image.resize(x, (IMG_HEIGHT, IMG_WIDTH)))
train_dataset = train_dataset.map(lambda x: x / 255.0)
train_dataset = train_dataset.map(lambda x: tf.image.rgb_to_grayscale(x))
train_dataset = train_dataset.batch(BATCH_SIZE)

## Visualize the data

In [ ]:
plt.figure(figsize=(10, 10))
for images in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().reshape(IMG_HEIGHT, IMG_WIDTH), cmap='gray')
        plt.axis('off')

## Optimize the datasets for performance

Cache, Shuffle, and Prefetch the Dataset

In [ ]:
# # train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# # test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# # remove labels
# ds = ds.map(lambda x: x["image"])
# # scale images
# ds = ds.map(lambda x: scale(x))
# ds = ds.cache()
# ds = ds.shuffle(1000)
# ds = ds.batch(128)
# ds = ds.prefetch(64)

### Define the Generator Model

In [ ]:
def build_generator():
    model = models.Sequential()
    
    # Convolutional layers for encoding
    model.add(layers.Conv2D(32, kernel_size=4, strides=1, padding='same', input_shape=[IMG_HEIGHT, IMG_WIDTH, 1]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2D(64, kernel_size=4, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2D(128, kernel_size=4, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=1, padding='same', activation='tanh'))
    return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
img = generator.predict(images)
plt.imshow(img[0, :, :, 0], cmap='gray')
plt.show()

In [ ]:
img.shape

In [ ]:
def build_discriminator():
    model = models.Sequential()
    
    # Convolutional layers for encoding
    model.add(layers.Conv2D(32, kernel_size=4, strides=2, padding='same', input_shape=[IMG_HEIGHT, IMG_WIDTH, 3]))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.4))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.4))
    
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.4))

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
decision = discriminator(img)
print(decision)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
# discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
generator_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
EPOCHS = 300
noise_dim = 100
num_examples_to_generate = 16
save_interval = 2
# You will reuse this seed overtime
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Training step function
@tf.function
def train_step(real_images):
    # Generate images using the generator
    noise = tf.random.normal([BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 1])  # Input noise or grayscale images
    generated_images = generator(noise, training=True)

    # Get the discriminator's output for real and fake images
    real_output = discriminator(real_images, training=True)
    fake_output = discriminator(generated_images, training=True)

    # Calculate the losses
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

    # Apply gradients to optimize the generator
    gradients_of_generator = tf.GradientTape().gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    # Apply gradients to optimize the discriminator
    gradients_of_discriminator = tf.GradientTape().gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def generate_and_save_images(generator, epoch, test_input, save_path='generated_images'):
    """
    Generate and save images from the Generator model.
    
    Parameters:
    - generator: The Generator model.
    - epoch: The current epoch number (for saving image with epoch information).
    - test_input: Random noise or grayscale images to generate images.
    - save_path: Directory path to save the generated images.
    """
    # Generate images
    predictions = generator(test_input, training=False)
    
    # Normalize images to the range [0, 1] for visualization
    predictions = (predictions + 1) / 2.0

    # Create a grid of images
    plt.figure(figsize=(10, 10))
    for i in range(predictions.shape[0]):
        plt.subplot(8, 8, i + 1)
        plt.imshow(predictions[i])
        plt.axis('off')

    # Save the generated images
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    plt.savefig(f'{save_path}/image_at_epoch_{epoch}.png')
    plt.close()

# Example usage:
# Generate and save images after each epoch
# generate_and_save_images(generator, epoch, test_input, save_path='generated_images')

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        if (epoch + 1) % save_interval == 0:
            # Generate and save images
            test_input = tf.random.normal([16, IMG_HEIGHT, IMG_WIDTH, 1])  # Generate some test images
            generate_and_save_images(generator, epoch + 1, test_input)

        print(f'Epoch {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')


In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
for image_batch in train_dataset.take(1):
    print(image_batch.shape)  # Should match [batch_size, IMG_HEIGHT, IMG_WIDTH, 3]
    plt.figure(figsize=(10, 10))
    for i in range(image_batch.shape[0]):
        plt.subplot(8, 8, i + 1)
        plt.imshow((image_batch[i] + 1) / 2.0)  # Assuming images are normalized to [-1, 1]
        plt.axis('off')
    plt.show()
    break